# Adaptive Multimodal Continuous Ant Colony Optimization

### Salman Khan   22i-1285
### Hassan Ismail 22i-1285

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

from cec2013.cec2013 import CEC2013


# Termination Criterion (Fixed Number of Iterations)
def termination_criterion():
    termination_criterion.counter += 1
    return termination_criterion.counter >= termination_criterion.max_iterations


termination_criterion.counter = 0
termination_criterion.max_iterations = 100  # Set the maximum number of iterations


# Algorithm 1

def crowd_clustering(population, cluster_size, f):
    population = population.copy()
    clusters = []
    while (len(population) > 0):
        # Randomly select a reference point
        referencePoint = population[np.random.randint(0, len(population), dtype=np.int32)]

        # Calculate doori from all points in population
        distances = [np.linalg.norm(ind - referencePoint) for ind in population]

        # Kam doori = good cluster neighbour
        nearest_index = np.argmin(distances)
        nearest_individual = population[nearest_index]

        # Sort must
        neighbors_indices = np.argsort(distances)[:cluster_size]
        crowd = [population[i] for i in neighbors_indices]

        # Remove the pehle se istemaal points
        for i in sorted(neighbors_indices, reverse=True):
            population = np.delete(population, i, axis=0)

        clusters.append(crowd)

    return clusters


# AMS Part
# Algorithm 2: Clustering for Speciation
def clustering_for_speciation(population, cluster_size):
    population = sorted(population, key=lambda x: x[-1], reverse=True)
    species = []

    while len(population) >= cluster_size:
        best_individual = population.pop(0)
        distances = [np.linalg.norm(ind[:-1] - best_individual[:-1]) for ind in population]
        neighbors_indices = np.argsort(distances)[:cluster_size - 1]
        species_group = [best_individual] + [population[i] for i in neighbors_indices]
        for i in sorted(neighbors_indices, reverse=True):
            del population[i]
        species.append(species_group)

    return species


"""
# Algorithm 4: Generate Solutions
def We_make_solutions(colonySize, niches, fitnessMin, fitnessMax, fitness_function):
    new_solutions = []
    for niche in niches:
        niche_fitness_min = min([sol[-1] for sol in niche])
        niche_fitness_max = max([sol[-1] for sol in niche])
        niche_std_dev = 1.0 / (fitnessMax - fitnessMin + 1e-10)

        for _ in range(colonySize // len(niches)):
            probabilities = [np.exp(sol[-1] - niche_fitness_min) for sol in niche]
            probabilities /= np.sum(probabilities)
            selected_index = np.random.choice(len(niche), p=probabilities)
            selected_solution = niche[selected_index][:-1]

            mean = selected_solution if np.random.rand() <= 0.5 else np.random.uniform(0, 30, len(selected_solution))
            std_dev = niche_std_dev
            new_solution = np.random.normal(loc=mean, scale=std_dev)

            value = fitness_function(new_solution[0])  # Use F1 fitness function
            new_solutions.append(np.append(new_solution, value))

    return new_solutions
"""


# Algorithm 4
def We_make_solutions(colonySize, niches, fitnessMin, fitnessMax, f):
    naway_solutions = []  # Initialize the list to store solutions and fitness values

    for niche in niches:
        niche_fitness_min = min([sol[-1] for sol in niche])
        niche_fitness_max = max([sol[-1] for sol in niche])
        niche_std_dev = 1.0 / (fitnessMax - fitnessMin + 1e-10)

        for _ in range(colonySize // len(niches)):
            # Roulette wheel selection
            probabilities = np.array([np.exp(sol[-1] - niche_fitness_min) for sol in niche])
            probabilities /= np.sum(probabilities)  # Normalize probabilities
            selected_index = np.random.choice(len(niche), p=probabilities)

            # Ignore the fitness value
            selected_solution = niche[selected_index][:-1]

            mean = selected_solution if np.random.rand() <= 0.5 else np.random.uniform(-1, 1, len(selected_solution))
            std_dev = niche_std_dev
            new_solution = np.random.normal(loc=mean, scale=std_dev)

            value = f.evaluate(new_solution)

            # Append new solution with fitness value as a tuple
            naway_solutions.append((new_solution, value))

    return naway_solutions


"""
# Algorithm 5: Adaptive Local Search
def adaptive_LS(seedSet, size, fitnessValues, std_, samples, fitness_function):
    seedSet = seedSet.copy()
    fitnessValues = fitnessValues.copy()

    min_fitness_val = min(fitnessValues)
    max_fitness_val = max(fitnessValues)
    flag = False
    fitness_range = 1
    if min_fitness_val <= 0:
        fitness_range = max_fitness_val + abs(min_fitness_val) + 1e-10
        flag = True

    probabilities = []
    for i in range(size):
        if flag:
            probabilities += [(fitnessValues[i] + abs(min_fitness_val) + 1e-10) / fitness_range]
        else:
            probabilities += fitnessValues[i] / max_fitness_val

    for i in range(size):
        if np.random.rand() <= probabilities[i]:
            for j in range(samples):
                gaussian_val = np.random.normal(seedSet[i], std_)
                gaussian_fitness = fitness_function(gaussian_val[0])  # Use F1 fitness function
                if fitnessValues[i] < gaussian_fitness:
                    seedSet[i] = gaussian_val
                    fitnessValues[i] = gaussian_fitness
    return seedSet, fitnessValues
"""


# Algorithm 5:
def adaptive_LS(seedSet, size, fitnessValues, std_, samples, f):
    seedSet = seedSet.copy()
    fitnessValues = fitnessValues.copy()

    min_fitness_val = min(fitnessValues)
    max_fitness_val = max(fitnessValues)
    flag = False
    fitness_range = 1
    if min_fitness_val <= 0:
        fitness_range = max_fitness_val + abs(min_fitness_val) + 1e-10
        flag = True

    probabilities = []  # List to store probabilities

    # Correcting how probabilities are populated
    for i in range(size):
        if flag:
            probabilities.append((fitnessValues[i] + abs(min_fitness_val) + 1e-10) / fitness_range)
        else:
            probabilities.append(fitnessValues[i] / max_fitness_val)

    # Ensure probabilities are correctly matched to the size of fitnessValues
    if len(probabilities) != size:
        raise ValueError(f"Length of probabilities {len(probabilities)} does not match size {size}")

    # Check if seedSet and fitnessValues are non-empty and properly sized
    if len(seedSet) != size or len(fitnessValues) != size:
        raise ValueError(
            f"Seed set or fitness values size mismatch: seedSet {len(seedSet)}, fitnessValues {len(fitnessValues)}, size {size}")

    for i in range(size):
        if np.random.rand() <= probabilities[i]:
            for j in range(samples):
                gaussian_val = np.random.normal(seedSet[i], std_)
                gaussian_fitness = f.evaluate(gaussian_val)
                if fitnessValues[i] < gaussian_fitness:
                    seedSet[i] = gaussian_val
                    fitnessValues[i] = gaussian_fitness

    return seedSet, fitnessValues


# Main Algorithm
# Algorithm 6 for ACO
def Lame_ACO(NP, G, delta, epsilon_set, f):
    global termination_criterion
    # Step 1
    popupation = np.zeros((NP[0], f.get_dimension()))
    for j in range(f.get_dimension()):
        popupation[:, j] = np.random.uniform(f.get_lbound(j), f.get_ubound(j), size=NP[0])

    archive_fitness = np.empty(NP[0])
    for i in range(NP[0]):
        archive_fitness[i] = f.evaluate(popupation[i])

    while termination_criterion():
        # Step 2: Obtain FSmax and FSmin in the archive
        FSmax = np.max(archive_fitness)
        FSmin = np.min(archive_fitness)

        # Step 3: Randomly select niching size NS from G
        NS = np.random.choice(G)

        # Step 4: Partition the archive into crowds using Algorithm 1
        niches = crowd_clustering(popupation, NS, f)  # Assuming we are using the first function for partitioning

        # Step 5: Construct new solutions using Algorithm 4
        colonySize = NP[0]  # Or you can sum NP for a total size: sum(NP)
        # print(f"Generated solution dimension: {len(new_solutions)}")
        new_solutions = We_make_solutions(colonySize, niches, FSmin, FSmax,
                                          f)  # Using the first fitness function for new solutions

        # continue changes from here
        # Step 6: Replace solutions in archive if new solution is better
        for new_solution in new_solutions:
            # Compare with nearest solution in archive
            distances = [np.linalg.norm(new_solution[:-1] - archive_ind[:-1]) for archive_ind in popupation]
            nearest_index = np.argmin(distances)
            if popupation[nearest_index, -1] > new_solution[-1]:  # If the new solution is better
                popupation[nearest_index] = new_solution

        # Step 7: Perform local search using Algorithm 5
        archive_solutions = popupation[:, :-1]
        # archive_fitness = archive[:, -1]
        archive_solutions, archive_fitness = adaptive_LS(archive_solutions, colonySize, archive_fitness, delta, 5, f)
        archive = np.column_stack((archive_solutions, archive_fitness))

        # Step 8: Termination check (You can add your own termination criteria)
        if np.max(archive_fitness) < epsilon_set[4]:  # Example termination condition using the smallest epsilon value
            break

    return popupation


# Algorithm 7 for AMS
def ant_colony_optimization(NP, G, delta, f):
    global termination_criterion  # Assume a global termination criterion is defined

    # Step 1: Randomly initialize NP solutions and evaluate their fitness
    archive = np.zeros((NP[0], f.get_dimension()))
    for j in range(f.get_dimension()):
        archive[:, j] = np.random.uniform(f.get_lbound(j), f.get_ubound(j), size=NP[0])

    archive_fitness = np.empty(NP[0])
    for i in range(NP[0]):
        archive_fitness[i] = f.evaluate(archive[i])

    while termination_criterion():
        # Step 2: Obtain FSmax and FSmin in the archive
        FSmax = max(archive_fitness)
        FSmin = min(archive_fitness)

        # Step 3: Randomly select a niching size NS from G
        NS = np.random.choice(G)

        # Step 4: Partition the archive into species using Algorithm 2
        population_with_fitness = [np.append(sol, fitness) for sol, fitness in zip(archive, archive_fitness)]
        species = clustering_for_speciation(population_with_fitness, NS)

        # Step 5: Construct NP solutions using Algorithm 4
        colonySize = NP[0]  # Or you can sum NP for a total size: sum(NP)
        new_solutions = We_make_solutions(colonySize, species, FSmin, FSmax, f)
        # new_solutions = We_make_solutions(NP, species, FSmin, FSmax, f)

        # Step 6: Compare and replace solutions in each species
        for species_group in species:
            for new_solution in new_solutions:
                distances = [np.linalg.norm(np.array(ind[:-1]) - np.array(new_solution[:-1])) for ind in species_group]
                nearest_idx = np.argmin(distances)
                nearest_solution = species_group[nearest_idx]

                if new_solution[-1] > nearest_solution[-1]:  # Compare fitness
                    species_group[nearest_idx] = new_solution

        # Update the archive with the new species
        archive = [sol[:-1] for species_group in species for sol in species_group]
        archive_fitness = [sol[-1] for species_group in species for sol in species_group]

        # Step 7: Perform local search using Algorithm 5
        archive, archive_fitness = adaptive_LS(archive, len(archive), archive_fitness, delta, 1, f)

    # Step 8: Return the archive
    return archive


def main():
    # Parameters for Ant Colony Optimization
    NP = [80, 80, 80, 80, 80, 100, 300, 300, 300, 100]
    G = [2, 4, 8, 12, 16, 20]
    delta = 1e-4
    epsilon_set = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]

    # Graphs for AMC
    print(f"Running AMC Ant Colony Optimization......")

    # Loop over fitness functions 1 to 9 from CEC2013
    for function_index in range(1, 11):  # Looping through CEC2013 functions 1-9
        print(f"Running Ant Colony Optimization for CEC2013 Function {function_index}")

        # Initialize fitness functions for the current function
        fitness_functions = CEC2013(function_index)

        # Run Ant Colony Optimization
        archive = Lame_ACO(NP, G, delta, epsilon_set, fitness_functions)

        # Capture the final solutions and their fitness values
        final_solutions = np.array(archive)
        fitness_values = [fitness_functions.evaluate(sol) for sol in final_solutions]

        # Handle different dimensions for plotting
        if final_solutions.shape[1] == 1:  # 1D
            plt.scatter(final_solutions[:, 0], fitness_values, c='blue', label='Solutions')
            plt.xlabel('Dimension 1')
            plt.ylabel('Fitness Value')
            plt.title(f'Final Solutions (1D) for CEC2013 Function {function_index}')
            plt.legend()
            plt.grid(True)
            plt.show()

        elif final_solutions.shape[1] == 2:  # 2D
            plt.scatter(final_solutions[:, 0], final_solutions[:, 1], c=fitness_values, cmap='viridis')
            plt.colorbar(label='Fitness Value')
            plt.xlabel('Dimension 1')
            plt.ylabel('Dimension 2')
            plt.title(f'Final Solutions (2D) for CEC2013 Function {function_index}')
            plt.show()

        elif final_solutions.shape[1] == 3:  # 3D
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            sc = ax.scatter(final_solutions[:, 0], final_solutions[:, 1], final_solutions[:, 2], c=fitness_values,
                            cmap='viridis')
            fig.colorbar(sc, label='Fitness Value')
            ax.set_xlabel('Dimension 1')
            ax.set_ylabel('Dimension 2')
            ax.set_zlabel('Dimension 3')
            ax.set_title(f'Final Solutions (3D) for CEC2013 Function {function_index}')
            plt.show()

        elif final_solutions.shape[1] > 3:  # More than 3 dimensions
            print(f"Reducing {final_solutions.shape[1]} dimensions to 3 using PCA for visualization.")
            pca = PCA(n_components=3)
            reduced_solutions = pca.fit_transform(final_solutions)
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            sc = ax.scatter(reduced_solutions[:, 0], reduced_solutions[:, 1], reduced_solutions[:, 2], c=fitness_values,
                            cmap='viridis')
            fig.colorbar(sc, label='Fitness Value')
            ax.set_xlabel('PCA Dimension 1')
            ax.set_ylabel('PCA Dimension 2')
            ax.set_zlabel('PCA Dimension 3')
            ax.set_title(f'Final Solutions (Reduced to 3D via PCA) for CEC2013 Function {function_index}')
            plt.show()
        else:
            print(f"Cannot plot less than 2 dimensions for CEC2013 Function {function_index}.")

    # Graphs for AMS
    print(f"Running AMS Ant Colony Optimization......")

    # Loop over fitness functions 1 to 8 from CEC2013
    for function_index in range(1, 11):  # Looping through CEC2013 functions 1-8
        print(f"Running Ant Colony Optimization for CEC2013 Function {function_index}")

        # Initialize fitness functions for the current function
        fitness_functions = CEC2013(function_index)

        # Run Ant Colony Optimization
        archive = ant_colony_optimization(NP, G, delta, fitness_functions)

        # Capture the final solutions and their fitness values
        final_solutions = np.array(archive)
        fitness_values = [fitness_functions.evaluate(sol) for sol in final_solutions]

        # Plot 1D, 2D, and 3D solutions
        if final_solutions.shape[1] == 1:  # 1D
            plt.scatter(final_solutions[:, 0], fitness_values, c='blue', label='Solutions')
            plt.xlabel('Dimension 1')
            plt.ylabel('Fitness Value')
            plt.title(f'Final Solutions (1D) for CEC2013 Function {function_index}')
            plt.legend()
            plt.grid(True)
            plt.show()

        elif final_solutions.shape[1] == 2:  # 2D
            plt.scatter(final_solutions[:, 0], final_solutions[:, 1], c=fitness_values, cmap='viridis')
            plt.colorbar(label='Fitness Value')
            plt.xlabel('Dimension 1')
            plt.ylabel('Dimension 2')
            plt.title(f'Final Solutions (2D) for CEC2013 Function {function_index}')
            plt.show()

        elif final_solutions.shape[1] == 3:  # 3D
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            sc = ax.scatter(final_solutions[:, 0], final_solutions[:, 1], final_solutions[:, 2], c=fitness_values,
                            cmap='viridis')
            fig.colorbar(sc, label='Fitness Value')
            ax.set_xlabel('Dimension 1')
            ax.set_ylabel('Dimension 2')
            ax.set_zlabel('Dimension 3')
            ax.set_title(f'Final Solutions (3D) for CEC2013 Function {function_index}')
            plt.show()

        else:  # More than 3 dimensions, reduce using PCA for visualization
            print(f"Reducing {final_solutions.shape[1]} dimensions to 3 using PCA for visualization.")
            pca = PCA(n_components=3)
            reduced_solutions = pca.fit_transform(final_solutions)
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            sc = ax.scatter(reduced_solutions[:, 0], reduced_solutions[:, 1], reduced_solutions[:, 2],
                            c=fitness_values,
                            cmap='viridis')
            fig.colorbar(sc, label='Fitness Value')
            ax.set_xlabel('PCA Dimension 1')
            ax.set_ylabel('PCA Dimension 2')
            ax.set_zlabel('PCA Dimension 3')
            ax.set_title(f'Final Solutions (Reduced to 3D via PCA) for CEC2013 Function {function_index}')
            plt.show()


# run main for AMC,AMS
if __name__ == "__main__":
    main()
